In [1]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

columns = ['data_inversa', 'mortos', 'municipio', 'uf', 'pessoas' ]
df = pd.DataFrame([], columns = columns)
for i,ano in enumerate(np.arange(2007,2022)):
    data = pd.read_csv("../Detram/datatran" + str(ano) + '.csv', delimiter = ';',  low_memory=False)
    df1 = data[columns]
    df = pd.concat([df,df1])

In [2]:
df = pd.DataFrame(df)
df['data_inversa'] = pd.to_datetime(df['data_inversa'])
df['pessoas'] = pd.to_numeric(df['pessoas'])
df['mortos'] = pd.to_numeric(df['mortos'])

df['acidentes'] = 1

n = df[df['uf'] =='(null)' ].index
df = df.drop(n)
df = df.rename({'data_inversa': 'Data', 'municipio': 'Municipio', 'pessoas': 'Pessoas', 'mortos': 'Mortos', 'acidentes': 'Acidente', 'uf': 'Estado'}, axis='columns')

#df.to_excel(r'./dataset.xlsx')
#data = pd.read_csv('./dataset.csv')

In [3]:
df['Ano'] = df['Data'].dt.year  

In [4]:
df['Mês'] = df['Data'].dt.strftime('%B')

In [5]:
df = df.sort_values(by=['Estado', 'Municipio', 'Ano', 'Mês']) 

# Acidentes de Trânsito 

 
<p style='text-align: justify;'> Os acidentes de trânsito geram muitas consequências para a população, como problemas econômicos, engarrafamentos quilométricos e várias mortes que pode acontecer em um único acidente. Portanto, necessitamos resolver sistematicamente esse problema procurando criar mecânicos preventivos, para isso, necessitamos saber primeiamente onde e quando devemos empenhar maior esforço nesse combate e uma das ferramentas que temos é no tratamento e visualização dos dados coletados. </p>

<p style='text-align: justify;'>No site do Detram obtemos vários dados coletados entre 2007 a 2021 e continua sendo atualizado constantemente, nos dados contém várias informações, como a ocorrência dos acidentes e mortes ocorridas nas rodovias brasileiras, esses dados são de grande importância para criar alternativas para a diminuição de acidentes e consequentemente na diminuição de óbitos nas estradas.</p>
    
<p style='text-align: justify;'>Com os dados em mãos, foram gerados três gráficos, uma sendo o gráfico de dispersão e duas sendo 0 grafico de barras empilhado. Na primeira visualização temos a correlação dos acidentes com as mortes no país de cada ano; a segunda visualização, mostra a quantidade de acidentes por estado que está empilhaods por mês para cada ano; por fim a terceira temos a visualização da quantidade de acidentes por ano empilhado por municipio para cada ano. 



In [6]:
df1 = df.groupby(by=['Ano','Estado']).agg('sum').reset_index()
dfcor = df1[['Acidente', 'Mortos']]

In [7]:
corr = dfcor.corr()


In [22]:
animations = {
          'Scatter': px.scatter(df1, x="Acidente", y="Mortos", animation_frame="Ano", animation_group="Ano", color = "Estado",
           size="Mortos",range_x=[0,max(df1['Acidente'].values)+300], range_y=[0,max(df1['Mortos'].values)+100],
           title="Diagrama de Dispersão de Acidentes por Mortes")}   

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.RadioItems(id = 'selection',
                 value = 'Scatter'),
    
    dcc.Graph(id="graph")
    
])
@app.callback(
    Output("graph", "figure"), 
    [Input("selection", "value")])

def display_animated_graph(s):
    
    return animations[s]

if __name__ == "__main__":
    app.run_server(debug=True, mode='inline', port = '8190')


In [11]:
print('Temos uma forte correlação entre Acidentes e Mortes ' + str(corr.values[0,1]))

Temos uma forte correlação entre Acidentes e Mortes 0.8954148652959313


<p style='text-align: justify;'> No gráfico de dispersão correlacionando acidentes com as mortes podemos destacar algumas observações:</p>

 <p>1. Nota-se uma diminuição gradativa nos valores ao passar do ano;</p>
 <p>2. O estado com maior índice de acidentes e mortes está no Estado de Minas Gerias em todos anos; </p>
 <p>3. Temos uma correlação forte de 0.897 entre Acidente e Mortes.</p>


In [14]:
df2 = df.groupby(by=['Estado','Ano', 'Mês']).agg('sum').reset_index()

In [24]:
Estados = df2.Estado.unique()

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id="selectstate",
        options=[{"label": x, "value": x} for x in Estados],
        placeholder = "Selecione o Estado"
    ),
    dcc.Graph(id="bar-chart"),
])

app = dash.Dash()                  
@app.callback(
    Output("bar-chart", "figure"), 
    [Input("selectstate", "value")])
def update_bar_chart(estado):
    fig = px.bar(df2[df2["Estado"] == estado], x="Ano", y="Acidente", title="Quantidade de Acidentes por Estado", color = 'Mês')
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, mode='inline', port = '8190')

TypeError: run_simple() got an unexpected keyword argument 'mode'

<p style='text-align: justify;'> No segundo gráfico de barras empilhado podemos selecionar o estado e obter algumas informações:</p>

   <p>1. Podemos visualizar em qual ano teve maior ocorrência de acidentes para cada estado;</p>
    <p>2. Analisar em qual mês teve as maiores ocorrência de acidentes para cada ano;</p> 
    <p style='text-align: justify;'> 3. Em Minas Gerais que obteve as maiores ocorrências de acidentes em todos os anos, podemos perceber que no ano de 2020 nos meses de janeiro, dezembro e fevereiro obtiveram a valores altos de acidentes em comparação com os outros meses. </p>


In [20]:
df3 = df.groupby(by=['Estado','Ano','Municipio']).agg('sum').reset_index()

In [21]:
Estados = df3.Estado.unique()


app.layout = html.Div([
    dcc.Dropdown(
        id="selectstate",
        options=[{"label": x, "value": x} for x in Estados], 
        placeholder = "Selecione o Estado"

    ),
    dcc.Graph(id="bar-chart")
])
app = dash.Dash()
@app.callback(
    Output("bar-chart", "figure"), 
    [Input("selectstate", "value")])
def update_bar_chart(estado):
    fig = px.bar(df3[df3["Estado"] == estado], x="Ano", y="Acidente", title="Quantidade de Acidentes por Municipio", color='Municipio')
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')



TypeError: run_simple() got an unexpected keyword argument 'mode'

<p style='text-align: justify;'> No último gráfico de barras empilhado podemos selecionar o estado  e obter algumas informações de cada município para cada ano:</p>

   <p>1. Podemos visualizar em qual ano teve maior ocorrência de acidentes em cada município selecionando o estado;</p>
   <p>2. Analisar em qual município teve maior ocorrência de acidentes para cada estado;</p>
   <p style='text-align: justify;'>3. Em Minas Gerais que obteve as maiores ocorrências de acidentes em todos os anos, podemos perceber que na cidade de Betim teve maior ocorrência de acidentes em todos os anos.</p> 


<p style='text-align: justify;'>Com base nas informações e nas visualizações coletadas, podemos dizer com convicção onde e em qual época devemos tomar medida para frear os acidentes nas estradas do país. Que através da análise, como por exemplo, podemos ter maior foco em Minas Gerias, na cidade de Betim, nos meses de dezembro, janeiro e fevereiro. </p>